In [1]:
import os 
import datetime 
import numpy as np
import pandas as pd 
import mplfinance as mpf
import matplotlib.pyplot as plt
import yfinance as yf
import time 

from ALGO_KT1 import Preprocessing_functions as pf 
from pathlib import Path

In [2]:
def perpetual_preferred(df1, ticker):

    symbol = df1[df1['Symbol'] == ticker]

    if symbol['time_to_maturity'].iloc[0] > 0:
        print('This is not a perpetual preferred yet!')

    price = pf.downlaod_symbol_data(ticker, period = "1day")
    
    # required rate of return calculated as r = annual_dividend / current price (aka current yield)
    r = np.round(float(symbol['Ann Amt'].iloc[0].replace('$', '')) / price['Close'].iloc[0], 4) 

    return  np.round(float(symbol['Ann Amt'].iloc[0].replace('$', '')) / r, 2 )


In [47]:
df1.head()

,Symbol,Security Description,StockExchange,IPO Date,Cpn Rate\nAnn Amt,LiqPref\nCallPrice,Call Date\nMatur Date,Moodys/S&PDated,15%\nTax Rate,Conv,IPO Prospectus,Distribution Dates,Call Date,Maturity Date,Cpn Rate,Ann Amt,time_to_maturity
1,ACP-PA,Aberdeen Income Credit Strategies Fund 5.250% ...,NYSE\nChart,05/03/2021,5.25% \n$1.3125,$25.00 \n$25.00,6/30/2026 \nNone,A2 NF \n3/19/2024,Variable,,Prospectus,"3/31, 6/30, 9/30 & 12/31\nClick for MW ExDiv D...",2026-06-30,None,5.25%,$1.3125,642
2,ACR-PC,ACRES Commercial Realty Corp 8.625% Fixed/Floa...,NYSE\nChart,06/03/2014,FixFloat \n$2.16,$25.00 \n$25.00,7/30/2024 \nNone,NR NF \n12/31/2023,No,,Prospectus,"1/30, 4/30, 7/30 & 10/30\nClick for MW ExDiv D...",2024-07-30,None,FixFloat,$2.16,-58
3,ACR-PD,"ACRES Commercial Realty Corp., 7.875% Series D...",NYSE\nChart,05/14/2021,7.88% \n$1.96875,$25.00 \n$25.00,5/21/2026 \nNone,NF NF \n12/31/2023,No,,Prospectus,"1/30, 4/30, 7/30 & 10/30\nClick for MW ExDiv D...",2026-05-21,None,7.88%,$1.96875,602
4,MITT-PB,"AG Mortgage Investment Trust, 8.00% Series B C...",NYSE\nChart,09/20/2012,8.00% \n$2.00,$25.00 \n$25.00,9/27/2017 \nNone,NF NF \n1/09/2023,No,,Prospectus,"3/17, 6/17, 9/17 & 12/17\nClick for MW ExDiv D...",2017-09-27,None,8.00%,$2.00,-2556
5,MITT-PC,"AG Mortgage Investment Trust, 8.00% Series C F...",NYSE\nChart,09/11/2019,FixFloat \n$2.00,$25.00 \n$25.00,9/17/2024 \nNone,NF NF \n1/09/2023,No,,Prospectus,"3/17, 6/17, 9/17 & 12/17\nClick for MW ExDiv D...",2024-09-17,None,FixFloat,$2.00,-9


In [48]:
pf.downlaod_symbol_data('ACR-PC', period = "1d")

,Open,High,Low,Close,Volume,Dividends,Stock Splits,open_low,open_close,open_high,high_low,low_close,high_close,gap
Date,,,,,,,,,,,,,,
2024-09-26 00:00:00-04:00,25.19,25.22,25.16,25.22,1880,0.0,0.0,0.119095,-0.119095,-0.119095,0.237906,-0.238474,0.0,NaN


In [69]:
ticker = 'JPM-PC'

today = datetime.date.today()

df = pf.downlaod_symbol_data(ticker, period = "6mo")

df1 = pd.read_csv('prefs_240924.csv')
df1 = df1.rename(columns={'Symbol\nCUSIP' : 'Symbol'})

df1['Symbol'] = df1['Symbol'].str.split('\n').apply(lambda x : x[0] if len(x) > 1 else x).str.replace('-', '-P')
df1 = df1[df1['Symbol'] != 'Symbol']
df1['Call Date'] = df1['Call Date\nMatur Date'].str.split('\n').apply(lambda x : x[0] if len(x) > 1 else x)
df1['Maturity Date'] = df1['Call Date\nMatur Date'].str.split('\n').apply(lambda x : x[1] if len(x) > 1 else x)
df1['Cpn Rate'] = df1['Cpn Rate\nAnn Amt'].str.split('\n').apply(lambda x : x[0] if len(x) > 1 else x)
df1['Ann Amt'] = df1['Cpn Rate\nAnn Amt'].str.split('\n').apply(lambda x : x[1] if len(x) > 1 else x)
df1 = df1[df1['Call Date'] != 'n.a. ']
df1 = df1[df1['Call Date'] != 'Called for']
today = pd.to_datetime(datetime.date.today())

df1['Call Date'] = pd.to_datetime(df1['Call Date'])

df1['time_to_maturity'] = (df1['Call Date'] - today).dt.days

df1.head()

,Symbol,Security Description,StockExchange,IPO Date,Cpn Rate\nAnn Amt,LiqPref\nCallPrice,Call Date\nMatur Date,Moodys/S&PDated,15%\nTax Rate,Conv,IPO Prospectus,Distribution Dates,Call Date,Maturity Date,Cpn Rate,Ann Amt,time_to_maturity
1,ACP-PA,Aberdeen Income Credit Strategies Fund 5.250% ...,NYSE\nChart,05/03/2021,5.25% \n$1.3125,$25.00 \n$25.00,6/30/2026 \nNone,A2 NF \n3/19/2024,Variable,,Prospectus,"3/31, 6/30, 9/30 & 12/31\nClick for MW ExDiv D...",2026-06-30,None,5.25%,$1.3125,642
2,ACR-PC,ACRES Commercial Realty Corp 8.625% Fixed/Floa...,NYSE\nChart,06/03/2014,FixFloat \n$2.16,$25.00 \n$25.00,7/30/2024 \nNone,NR NF \n12/31/2023,No,,Prospectus,"1/30, 4/30, 7/30 & 10/30\nClick for MW ExDiv D...",2024-07-30,None,FixFloat,$2.16,-58
3,ACR-PD,"ACRES Commercial Realty Corp., 7.875% Series D...",NYSE\nChart,05/14/2021,7.88% \n$1.96875,$25.00 \n$25.00,5/21/2026 \nNone,NF NF \n12/31/2023,No,,Prospectus,"1/30, 4/30, 7/30 & 10/30\nClick for MW ExDiv D...",2026-05-21,None,7.88%,$1.96875,602
4,MITT-PB,"AG Mortgage Investment Trust, 8.00% Series B C...",NYSE\nChart,09/20/2012,8.00% \n$2.00,$25.00 \n$25.00,9/27/2017 \nNone,NF NF \n1/09/2023,No,,Prospectus,"3/17, 6/17, 9/17 & 12/17\nClick for MW ExDiv D...",2017-09-27,None,8.00%,$2.00,-2556
5,MITT-PC,"AG Mortgage Investment Trust, 8.00% Series C F...",NYSE\nChart,09/11/2019,FixFloat \n$2.00,$25.00 \n$25.00,9/17/2024 \nNone,NF NF \n1/09/2023,No,,Prospectus,"3/17, 6/17, 9/17 & 12/17\nClick for MW ExDiv D...",2024-09-17,None,FixFloat,$2.00,-9


In [71]:
#%%capture ### supresses output for downloading data

prices = {}

symbols = df1.Symbol.dropna().to_list()

for symbol in symbols:

    try:
        try:
            #price = yf.download(symbol, today)['Adj Close'].iloc[0]
            price = yf.Ticker(symbol).history(period = '1d')['Close'].iloc[0]
        except IndexError:
            continue 
    except KeyError:
        print(f"No data for symbol - {symbol}")
    
    prices[symbol] = price    

prices = pd.DataFrame([prices.keys(), prices.values()]).transpose()
prices.columns = ['Symbol' , 'price']

df1 = df1.merge(prices, on = 'Symbol')

ACP-PA: No price data found, symbol may be delisted (period=1d)
AATRL: No price data found, symbol may be delisted (period=1d)
AVTR-PA: No data found, symbol may be delisted
TEMP85: No data found, symbol may be delisted
TEMP05: No data found, symbol may be delisted
TEMP46: No data found, symbol may be delisted
BKNML: No data found, symbol may be delisted
BCPPF: No price data found, symbol may be delisted (period=1d)
BCEPF: No price data found, symbol may be delisted (period=1d)
BKFPF: No data found, symbol may be delisted
BKAAF: No price data found, symbol may be delisted (period=1d)
CWESF: Period '1d' is invalid, must be one of ['1mo', '3mo', '6mo', 'ytd', '1y', '2y', '5y', '10y', 'max']
CPWPF: No price data found, symbol may be delisted (period=1d)
TEMP52: No data found, symbol may be delisted
CSSEP: No price data found, symbol may be delisted (period=1d)
CBKAP: No data found, symbol may be delisted
CBCPQ: No data found, symbol may be delisted
CYCCP: No price data found, symbol may b

In [75]:
df1['Ann Amt'] = df1['Ann Amt'].apply(lambda x : x.replace('$', ''))
df1['Ann Amt'] = df1['Ann Amt'].apply(lambda x : x.replace('n.a.', '0'))

Ann Amt
0            26
2.00         16
1.25         15
1.75         15
1.41         12
             ..
1.075         1
1.464475      1
1.67076       1
1.675925      1
2.09375       1
Name: count, Length: 171, dtype: int64

In [78]:
df1['Ann Amt'] = df1['Ann Amt'].astype(float)
df1['price'] = df1['price'].astype(float)
df1['current_yield'] = df1['Ann Amt'] / df1['price']


df1['perpetual_valuation'] = df1['Ann Amt'] / df1['r'] # we have to figure out the required rate of return 
df1.head()

,Symbol,Security Description,StockExchange,IPO Date,Cpn Rate\nAnn Amt,LiqPref\nCallPrice,Call Date\nMatur Date,Moodys/S&PDated,15%\nTax Rate,Conv,...,Distribution Dates,Call Date,Maturity Date,Cpn Rate,Ann Amt,time_to_maturity,price,Ann Amt1,r,perpetual_valuation
0,ACR-PC,ACRES Commercial Realty Corp 8.625% Fixed/Floa...,NYSE\nChart,06/03/2014,FixFloat \n$2.16,$25.00 \n$25.00,7/30/2024 \nNone,NR NF \n12/31/2023,No,,...,"1/30, 4/30, 7/30 & 10/30\nClick for MW ExDiv D...",2024-07-30,None,FixFloat,2.16000,-58,25.209999,2.16,0.085680,25.209999
1,ACR-PD,"ACRES Commercial Realty Corp., 7.875% Series D...",NYSE\nChart,05/14/2021,7.88% \n$1.96875,$25.00 \n$25.00,5/21/2026 \nNone,NF NF \n12/31/2023,No,,...,"1/30, 4/30, 7/30 & 10/30\nClick for MW ExDiv D...",2026-05-21,None,7.88%,1.96875,602,23.400000,1.96875,0.084135,23.400000
2,MITT-PB,"AG Mortgage Investment Trust, 8.00% Series B C...",NYSE\nChart,09/20/2012,8.00% \n$2.00,$25.00 \n$25.00,9/27/2017 \nNone,NF NF \n1/09/2023,No,,...,"3/17, 6/17, 9/17 & 12/17\nClick for MW ExDiv D...",2017-09-27,None,8.00%,2.00000,-2556,22.200001,2.00,0.090090,22.200001
3,MITT-PC,"AG Mortgage Investment Trust, 8.00% Series C F...",NYSE\nChart,09/11/2019,FixFloat \n$2.00,$25.00 \n$25.00,9/17/2024 \nNone,NF NF \n1/09/2023,No,,...,"3/17, 6/17, 9/17 & 12/17\nClick for MW ExDiv D...",2024-09-17,None,FixFloat,2.00000,-9,24.850000,2.00,0.080483,24.850000
4,MITT-PA,"AG Mortgage Investment Trust, 8.25% Series A C...",NYSE\nChart,07/27/2012,8.25% \n$2.06,$25.00 \n$25.00,8/03/2017 \nNone,NF NF \n1/09/2023,No,,...,"3/17, 6/17, 9/17 & 12/17\nClick for MW ExDiv D...",2017-08-03,None,8.25%,2.06000,-2611,22.570000,2.06,0.091272,22.570000


In [76]:
today = datetime.date.today()
type(today)

datetime.date

In [75]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 652 entries, 1 to 803
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Symbol                613 non-null    object        
 1   Security Description  652 non-null    object        
 2   StockExchange         640 non-null    object        
 3   IPO Date              652 non-null    object        
 4   Cpn Rate
Ann Amt      652 non-null    object        
 5   LiqPref
CallPrice     652 non-null    object        
 6   Call Date
Matur Date  652 non-null    object        
 7   Moodys/S&PDated       652 non-null    object        
 8   15%
Tax Rate          652 non-null    object        
 9   Conv                  652 non-null    object        
 10  IPO Prospectus        652 non-null    object        
 11  Distribution Dates    652 non-null    object        
 12  Call Date             652 non-null    datetime64[ns]
 13  Maturity Date         652

In [43]:
# Convertible shares 

conv_M = df1[df1['Conv'].str.contains('M')]
conv_C = df1[df1['Conv'].str.contains('C')]

conv = pd.concat([conv_C, conv_M]).reset_index(drop= True)

conv.tail()

,Symbol,Security Description,StockExchange,IPO Date,Cpn Rate\nAnn Amt,LiqPref\nCallPrice,Call Date\nMatur Date,Moodys/S&PDated,15%\nTax Rate,Conv,IPO Prospectus,Distribution Dates,Call Date,Maturity Date,Cpn Rate,Ann Amt
40,HEWPV,Hewlett Packard Enterprise Co 7.625% Ser C Man...,OTOTC\nChart,09/10/2024,7.63% \n$3.8125,$50.00 \n$50.00,n.a. \n9/01/2027,NR NR \n9/11/2024,YES,M,Prospectus,"3/1, 6/1, 9/31& 12/1\nClick for MW ExDiv Date\...",n.a.,9/01/2027,7.63%,$3.8125
41,NYCB-PU,"New York Community Capital Trust V, BONUSES Un...",NYSE\nChart,10/30/2002,6.00% \n$3.00,$50.00 \nn.a.,n.a. \n10/01/2051,WR NF \n4/21/2024,No,M,Prospectus,"2/1, 5/1, 8/1 & 11/1\nClick for MW ExDiv Date\...",n.a.,10/01/2051,6.00%,$3.00
42,NEE-PR,"NextEra Energy, Inc. 6.926% Equity Units Due 0...",NYSE\nChart,09/14/2022,6.93% \n$3.463,$50.00 \nn.a.,n.a. \n9/01/2025,Baa2 BBB \n4/23/2023,No,M,Prospectus,"3/1, 6/1, 9/1 & 12/1\nClick for MW ExDiv Date\...",n.a.,9/01/2025,6.93%,$3.463
43,NEE-PS,"NextEra Energy, Inc. 7.229% Equity Units Due 6...",NYSE\nChart,06/17/2024,7.23% \n$3.6145,$50.00 \n$50.00,n.a. \n6/01/2027,NR NR \n6/26/2024,No,M,Prospectus,"3/1, 6/1, 9/1 & 12/1\nClick for MW ExDiv Date\...",n.a.,6/01/2027,7.23%,$3.6145
44,RBCP,RBC Bearings Inc 5.0% Ser A Mandatory Converti...,NYSE\nChart,09/20/2021,5.00% \n$5,$100.00 \n$100.00,n.a. \n10/15/2024,NF NR \n1/06/2023,YES,M,Prospectus,"1/15, 4/15, 7/15 & 10/15\nClick for MW ExDiv D...",n.a.,10/15/2024,5.00%,$5
